# Data Preparation


### imports

In [2]:
!pip install datasets -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [3]:
import pandas as pd
from datasets import DatasetDict, Dataset

### data prep

In [4]:
# data downloaded from here: https://www.kaggle.com/datasets/taruntiwarihp/phishing-site-urls/data
df = pd.read_csv("/content/phishing_site_urls.csv")

In [6]:
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [9]:
df.describe(include='all')

,URL,Label
count,549346,549346
unique,507195,2
top,jhomitevd2abj3fk.onion.to/,good
freq,52,392924


In [10]:
# drop data
df = df.dropna()

# create dataframes from each class

df_safe = df[df['Label']=="good"]
df_not_safe = df[df['Label']=="bad"]

# define number of samples to keep
num_samples = 10500

# Sample min_size rows from each class to ensure a 50-50 split
df_safe_sample = df_safe.sample(num_samples, random_state=42)
df_not_safe_sample = df_not_safe.sample(num_samples, random_state=42)

# replace "Email Type" with Boolean flag "isPhising"
df_safe_sample = df_safe_sample.assign(isPhishing=False)
df_safe_sample = df_safe_sample.drop('Label',axis=1)
df_not_safe_sample = df_not_safe_sample.assign(isPhishing=True)
df_not_safe_sample = df_not_safe_sample.drop('Label',axis=1)

# Concatenate the samples to create a new balanced dataset
balanced_df = pd.concat([df_safe_sample, df_not_safe_sample])
balanced_df.columns = ['text', 'labels']

# convert labels column to int
balanced_df['labels'] = balanced_df['labels'].astype(int)

# Shuffle the balanced dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into train, validation, and test sets (e.g., 70% train, 15% validation, 15% test)
train_frac = 0.7
valid_frac = 0.15
test_frac = 0.15

# define train and validation size
train_size = int(train_frac * len(balanced_df))
valid_size = int(valid_frac * len(balanced_df))

# create train, validation, and test datasets
train_df = balanced_df[:train_size]
valid_df = balanced_df[train_size:train_size + valid_size]
test_df = balanced_df[train_size + valid_size:]

# Convert the pandas DataFrames back to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_ds,
    'validation': valid_ds,
    'test': test_ds
})

In [11]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 14699
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3151
    })
})

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
# push data to hub
dataset_dict.push_to_hub("KaushiGihan/phishing-site-url")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/KaushiGihan/phishing-site-url/commit/af2885d17d912067256cbe27a3f6ab6b31c22694', commit_message='Upload dataset', commit_description='', oid='af2885d17d912067256cbe27a3f6ab6b31c22694', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KaushiGihan/phishing-site-url', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KaushiGihan/phishing-site-url'), pr_revision=None, pr_num=None)